In [29]:
import pandas as pd
import importlib

### Create job with colfax for W2V hyper-parameterization or hyper-tuning

In [1]:
import cfxmagic

In [5]:
%%writefile w2v_best_params.py
import Word2Vec as w2v
import os
import pandas as pd

data = w2v.ready_data('all')

questions = 'questions-words.txt'
file = os.path.join('data',questions)
evals = open(file,'r').readlines()
num_sections = len([l for l in evals if l.startswith(':')])
print('Total Evaluation sentences from questions-words.txt: {}'.format(len(evals) - num_sections))

#define parameter grid for Word2Vec
param_grid = {'size': [100,200,300,400],
            'alpha' : [0.05,0.025],
            'window' : [4,10],
            'workers' : [12],
            'min_count':[10],
            'sg': [0,1],
            'hs': [1], 
            'negative': [5,10]
}

#run the exhaustive grid_search on param_grid
results = w2v.run_w2v_hypertuning(data,param_grid)

pd.DataFrame(results)

Overwriting w2v_best_params.py


In [6]:
%%qsub -N w2v_best_params -l walltime=24:00:00 -l nodes=2:ppn=2
cd $PBS_O_WORKDIR
python w2v_best_params.py

185777.c009



In [1]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
186691.c009                ...ub-singleuser u18634          00:00:01 R jupyterhub     


In [3]:
%ls w2v_best_params*

w2v_best_params.e179485  w2v_best_params.o179485  w2v_best_params.py
w2v_best_params.e181279  w2v_best_params.o181279
w2v_best_params.e184059  w2v_best_params.o184059


In [5]:
%cat w2v_best_params.o184059*


########################################################################
#      Date:           Tue Oct  9 23:52:51 PDT 2018
#    Job ID:           184059.c009
#      User:           u18634
# Resources:           neednodes=2:ppn=2,nodes=2:ppn=2,vmem=92gb,walltime=24:00:00
########################################################################

Total Evaluation sentences from questions-words.txt: 19544

########################################################################
# End of output for job 184059.c009
# Date: Wed Oct 10 13:55:04 PDT 2018
########################################################################



In [6]:
%cat w2v_best_params.e184059*

/home/u18634/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [3]:
%cat w2v_best_params.o185777*


########################################################################
#      Date:           Sun Oct 14 23:42:27 PDT 2018
#    Job ID:           185777.c009
#      User:           u18634
# Resources:           neednodes=2:ppn=2,nodes=2:ppn=2,vmem=92gb,walltime=24:00:00
########################################################################


########################################################################
# End of output for job 185777.c009
# Date: Mon Oct 15 23:43:06 PDT 2018
########################################################################



In [2]:
%cat w2v_best_params.e185777*

=>> PBS: job killed: walltime 86439 exceeded limit 86400


In [9]:
#read the results from stored pickles
import os
import pandas as pd

grid_result = pd.read_pickle(os.path.join('data','w2v_grid_search_results.pkl'))
grid_result.sort_values('correct_per',ascending=False).head(10)

,alpha,correct_per,hs,incorrect_per,min_count,negative,sg,size,window,workers
36,0.025,37.827487,0,62.172513,10,5,0,200,2,12
60,0.025,37.801375,0,62.198625,10,5,0,300,2,12
61,0.025,37.723039,0,62.276961,10,10,0,300,2,12
37,0.025,37.470624,0,62.529376,10,10,0,200,2,12
66,0.025,37.357472,0,62.642528,10,5,0,300,4,12
85,0.025,37.279137,0,62.720863,10,10,0,400,2,12
62,0.025,37.096353,0,62.903647,10,20,0,300,2,12
84,0.025,37.035425,0,62.964575,10,5,0,400,2,12
38,0.025,37.018017,0,62.981983,10,20,0,200,2,12
43,0.025,36.965793,0,63.034207,10,10,0,200,4,12
